In [59]:
import pandas as pd

df_cluster = pd.read_csv('subscribers_cleaned_dummified.csv').fillna(0)
convert_TF = []
for i in range(0, 63417):
    if df_cluster.iloc[i]['payment_period'] == 0.0 and df_cluster.iloc[i]['current_sub_TF'] == 1.0:
        convert_TF.append('Check')
    elif df_cluster.iloc[i]['payment_period'] == 0.0 and df_cluster.iloc[i]['current_sub_TF'] == 0.0:
        convert_TF.append(False)
    else:
        convert_TF.append(True)

df_cluster['convert_TF'] = convert_TF

df_attri = df_cluster[(df_cluster['attribution_technical_bing'] == 1.0)
                      | (df_cluster['attribution_technical_display'] == 1.0)
                      | (df_cluster['attribution_technical_facebook'] == 1.0)
                      | (df_cluster['attribution_technical_search'] == 1.0)
                      | (df_cluster['attribution_technical_youtube'] == 1.0)]


print(df_attri)

          subid  weekly_consumption_hour   age  male_TF  monthly_price  \
0      29417030                20.051667  30.0        0         4.7343   
1      29726122                20.051667  61.0        0         4.7343   
2      24930568                25.851492  73.0        0         4.7343   
3      23452753                28.751405  71.0        0         4.7343   
4      21361112                37.451142  24.0        1         4.7343   
5      23163827                33.101274  54.0        0         4.7343   
6      27648682                27.301448  26.0        0         4.7343   
7      29090912                37.451142  50.0        0         4.7343   
8      21447938                30.201361  59.0        0         4.7343   
9      21908764                25.851492  29.0        0         4.7343   
10     29615706                25.851492  61.0        0         4.7343   
11     27843327                27.301448  48.0        0         4.7343   
12     26461912                25.8514

In [60]:
df_attri = df_attri.loc[:, ['subid', 'tier','convert_TF','attribution_technical_bing', 'attribution_technical_display', 
                 'attribution_technical_facebook', 'attribution_technical_youtube', 'attribution_technical_search',
                           'join_fee', 'monthly_price', 'payment_period']]

In [84]:
attribution_technical = list()
for i in range(len(df_attri)):
    if df_attri.loc[i,:][3] == 1:
        attribution_technical.append('bing')
    elif df_attri.loc[i,:][4] == 1:
        attribution_technical.append('display')
    elif df_attri.loc[i,:][5] == 1:
        attribution_technical.append('facebook')
    elif df_attri.loc[i,:][6] == 1:
        attribution_technical.append('youtube')
    else: 
        attribution_technical.append('search')

In [85]:
df_attri['attribution_technical'] = attribution_technical
df_attri

,subid,tier,convert_TF,attribution_technical_bing,attribution_technical_display,attribution_technical_facebook,attribution_technical_youtube,attribution_technical_search,join_fee,monthly_price,payment_period,attribution_technical
0,29417030,7,False,0,0,0,0,1,0.0367,4.7343,0,search
1,29726122,8,True,1,0,0,0,0,0.3303,4.7343,1,bing
2,24930568,5,True,0,0,1,0,0,0.6606,4.7343,1,facebook
3,23452753,7,False,0,0,0,0,1,0.3303,4.7343,0,search
4,21361112,6,True,0,0,0,0,1,0.3303,4.7343,1,search
5,23163827,4,False,0,0,1,0,0,0.0367,4.7343,0,facebook
6,27648682,4,True,0,0,0,0,1,0.0367,4.7343,2,search
7,29090912,3,True,0,0,1,0,0,0.0367,4.7343,1,facebook
8,21447938,6,False,0,1,0,0,0,0.3303,4.7343,0,display
9,21908764,3,True,0,0,1,0,0,0.0367,4.7343,1,facebook


In [86]:
import json
channel_spend_dict = {}
channel_spend = pd.read_csv('channel_spend_undergraduate.csv')
tier_list = ['tier1', 'tier2', 'tier3','tier4', 'tier5', 'tier6','tier7', 'tier8']
for i in range(8):
    channel_spend_dict[tier_list[i]] = json.loads(channel_spend['spend'][i].replace("'", '"'))
# channel_spend_dict
channel_spend = channel_spend_dict

In [87]:
channel_spend['tier1'].keys()

dict_keys(['bing', 'display', 'facebook', 'search', 'youtube'])

In [88]:
# ----- Set parameters -----
touch_col_prepend = 'touch'
direct_label = 'direct'
first_weight = 0.4
last_weight = 0.4
cred_col_post_pend = '_credit'
write_to_file = True

# total spending for all 8 tier experiments
channel_spend['total'] = dict()
for t_name, t in channel_spend.items():
    if t_name != 'total':
        for c in t.keys():
            try:
                channel_spend['total'][c] = channel_spend['total'][c] + t[c]
            except KeyError:
                channel_spend['total'].update({c: t[c]})

In [89]:
channel_spend

{'tier1': {'bing': 300,
  'display': 12,
  'facebook': 9000,
  'search': 13000,
  'youtube': 90},
 'tier2': {'bing': 400,
  'display': 13,
  'facebook': 10500,
  'search': 18500,
  'youtube': 100},
 'tier3': {'bing': 900,
  'display': 19,
  'facebook': 11000,
  'search': 19000,
  'youtube': 130},
 'tier4': {'bing': 1000,
  'display': 20,
  'facebook': 13000,
  'search': 24000,
  'youtube': 180},
 'tier5': {'bing': 1100,
  'display': 29,
  'facebook': 14000,
  'search': 25000,
  'youtube': 550},
 'tier6': {'bing': 1300,
  'display': 31,
  'facebook': 16000,
  'search': 38000,
  'youtube': 900},
 'tier7': {'bing': 2100,
  'display': 94,
  'facebook': 17000,
  'search': 41000,
  'youtube': 2420},
 'tier8': {'bing': 3700,
  'display': 148,
  'facebook': 23000,
  'search': 44000,
  'youtube': 4360},
 'total': {'bing': 10800,
  'display': 366,
  'facebook': 113500,
  'search': 222500,
  'youtube': 8730}}

In [90]:
# ----- Format dataframe -----
# --- create credit columns
cred_col_names = ['bing_credit','display_credit','facebook_credit','search_credit','youtube_credit']

In [91]:
print(df_attri[df_attri['attribution_technical_facebook'] == 1].shape)
print(df_attri[df_attri['attribution_technical_bing'] == 1].shape)
print(df_attri[df_attri['attribution_technical_display'] == 1].shape)
print(df_attri[df_attri['attribution_technical_search'] == 1].shape)
print(df_attri[df_attri['attribution_technical_youtube'] == 1].shape)

(43571, 12)
(804, 12)
(1113, 12)
(17236, 12)
(693, 12)


In [95]:
def assign_credit(t_row, cred_col_names_f, cred_col_post_pend_f, model_type_f, first_weight_f=0.5, last_weight_f=0.5):
    # function assigns a credit to each relevant channel based on user specified model type, e.g. "last_touch_point", "first_touch_point", etc.
    t_dict = dict(zip(cred_col_names_f, [0]*len(cred_col_names_f)))

    if model_type_f == 'attribution_technical':
        if t_row['attribution_technical'] == 'bing' or t_row['attribution_technical'] == 'display'or t_row['attribution_technical'] ==  'facebook' or t_row['attribution_technical'] == 'search' or t_row['attribution_technical'] == 'youtube':
            t_dict.update({t_row['attribution_technical'] + cred_col_post_pend_f: 1})
        return t_dict
    
def get_attribution_by_channel(df_f, credit_col_postpend_f):
    allocated_conversions = df_f[cred_col_names].sum()
    print(allocated_conversions)
    n_allocated_conversions = df_f[cred_col_names].sum().sum()
    print(n_allocated_conversions)
    n_total_conversions = df_f['convert_TF'].sum()
    channel_allocation_f = pd.Series(dict(zip([x.split(credit_col_postpend_f)[0] for x in allocated_conversions.keys()], list(allocated_conversions.array))))
    return channel_allocation_f

def calc_avg_CAC(channel_allocation_f, channel_spend_f):
    t_df = pd.DataFrame(channel_allocation_f)
    t_df.columns = ['channel_allocation']
    for t_ind, _ in t_df.iterrows():
        t_df.loc[t_ind, 'channel_spend'] = channel_spend_f[t_ind]

    t_df['CAC'] = t_df['channel_spend'] / t_df['channel_allocation']
    t_df['CAC'].replace(np.inf, 0, inplace=True)
    return t_df

def calc_marginal_CAC(n_conversions_low_tier, spend_low_tier, n_conversions_high_tier, spend_high_tier):
    ##### fill in this code to create the three variables in output dictionary
    marginal_conversions = n_conversions_high_tier - n_conversions_low_tier
    marginal_spend = spend_high_tier - spend_low_tier
    marginal_CAC = marginal_spend/marginal_conversions
    return {'marginal_conversions': marginal_conversions, 'marginal_spend': marginal_spend,
            'marginal_CAC': marginal_CAC}

In [98]:
df_attri.convert_TF = df_attri.convert_TF.fillna(False)
import numpy as np

In [116]:
# ----- RUN MODELS -----
CAC_dfs = dict()
model_type = 'attribution_technical'
print('Processing model %s' % model_type)

# ----- Run attribution model -----
print('Running attribution model')
df_convert = df_attri.loc[df_attri['convert_TF']==True] # only run calculation for conversion rows
info_to_add = list()
for t_ind, t_row in df_convert.iterrows():
    t_credit_dict = assign_credit(t_row, cred_col_names, cred_col_post_pend, model_type, first_weight, last_weight)
    info_to_add.append(t_credit_dict)
df_convert = pd.concat([df_convert.reset_index(drop=True), pd.DataFrame(info_to_add).reset_index(drop=True)], axis=1)

# ----- Calculate CAC -----
print('Calculating average and marginal CAC')
# --- Average CAC ---
channel_allocation = get_attribution_by_channel(df_convert, credit_col_postpend_f='_credit')
df_CAC = calc_avg_CAC(channel_allocation, channel_spend['total'])
print('Average:') 
print(df_CAC)
df_CAC.to_csv('average_cac_technical.csv')

# --- Marginal CAC ---
credit_cols = [x for x in df_convert.columns if x.find('credit') > -1]
df_CAC = pd.DataFrame(index=[x.split('_credit')[0] for x in credit_cols])
base_col_names = ['marginal_conversions', 'marginal_spend', 'marginal_CAC']

df_tier_sum = df_convert[['tier']+credit_cols].groupby(['tier']).sum()
df_tier_sum.columns = [x.split('_credit')[0] for x in df_tier_sum.columns]
for t_tier in df_tier_sum.index:
    for t_channel in df_CAC.index:
        if t_tier > 1:
            n_conversions_low_tier = df_tier_sum.loc[t_tier - 1, t_channel]
            spend_low_tier = channel_spend['tier' + str(t_tier - 1)][t_channel]
            n_conversions_high_tier = df_tier_sum.loc[t_tier, t_channel]
            spend_high_tier = channel_spend['tier' + str(t_tier)][t_channel]
        else:
            n_conversions_low_tier = 0
            spend_low_tier = 0
            n_conversions_high_tier = df_tier_sum.loc[t_tier, t_channel]
            spend_high_tier = channel_spend['tier' + str(t_tier)][t_channel]

        t_df_CAC_colnames = [x + '_t' + str(t_tier) for x in base_col_names]
        for i in t_df_CAC_colnames:
            if i not in list(df_CAC.columns):
                df_CAC[i] = float('nan')

        t_marginal_dict = calc_marginal_CAC(n_conversions_low_tier, spend_low_tier, n_conversions_high_tier, spend_high_tier)
        df_CAC.loc[t_channel, t_df_CAC_colnames] = [t_marginal_dict[x] for x in base_col_names]
CAC_dfs.update({model_type: df_CAC})

Processing model attribution_technical
Running attribution model
Calculating average and marginal CAC
bing_credit          469
display_credit       679
facebook_credit    27340
search_credit      12181
youtube_credit       567
dtype: int64
41236
Average:
          channel_allocation  channel_spend        CAC
bing                     469        10800.0  23.027719
display                  679          366.0   0.539028
facebook               27340       113500.0   4.151426
search                 12181       222500.0  18.266152
youtube                  567         8730.0  15.396825


/Users/apple/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in long_scalars


In [119]:
CAC_dfs

{'attribution_technical':           marginal_conversions_t1  marginal_spend_t1  marginal_CAC_t1  \
 bing                         12.0              300.0        25.000000   
 display                       1.0               12.0        12.000000   
 facebook                    177.0             9000.0        50.847458   
 search                       43.0            13000.0       302.325581   
 youtube                       0.0               90.0              inf   
 
           marginal_conversions_t2  marginal_spend_t2  marginal_CAC_t2  \
 bing                         -9.0              100.0       -11.111111   
 display                       6.0                1.0         0.166667   
 facebook                   2542.0             1500.0         0.590087   
 search                      826.0             5500.0         6.658596   
 youtube                       1.0               10.0        10.000000   
 
           marginal_conversions_t3  marginal_spend_t3  marginal_CAC_t3  \
 bing    

In [120]:
CAC_dfs['attribution_technical'].loc[['bing', 'display', 'facebook', 'search', 'youtube']]

,marginal_conversions_t1,marginal_spend_t1,marginal_CAC_t1,marginal_conversions_t2,marginal_spend_t2,marginal_CAC_t2,marginal_conversions_t3,marginal_spend_t3,marginal_CAC_t3,marginal_conversions_t4,...,marginal_CAC_t5,marginal_conversions_t6,marginal_spend_t6,marginal_CAC_t6,marginal_conversions_t7,marginal_spend_t7,marginal_CAC_t7,marginal_conversions_t8,marginal_spend_t8,marginal_CAC_t8
bing,12.0,300.0,25.000000,-9.0,100.0,-11.111111,24.0,500.0,20.833333,24.0,...,50.000000,5.0,200.0,40.000000,34.0,800.0,23.529412,81.0,1600.0,19.753086
display,1.0,12.0,12.000000,6.0,1.0,0.166667,21.0,6.0,0.285714,-2.0,...,0.250000,2.0,2.0,1.000000,110.0,63.0,0.572727,143.0,54.0,0.377622
facebook,177.0,9000.0,50.847458,2542.0,1500.0,0.590087,197.0,500.0,2.538071,229.0,...,3.378378,871.0,2000.0,2.296211,313.0,1000.0,3.194888,1380.0,6000.0,4.347826
search,43.0,13000.0,302.325581,826.0,5500.0,6.658596,238.0,500.0,2.100840,323.0,...,7.751938,730.0,13000.0,17.808219,121.0,3000.0,24.793388,64.0,3000.0,46.875000
youtube,0.0,90.0,inf,1.0,10.0,10.000000,6.0,30.0,5.000000,1.0,...,13.214286,18.0,350.0,19.444444,99.0,1520.0,15.353535,155.0,1940.0,12.516129


In [121]:
# write marginal CAC output
if write_to_file:
    for key, value in CAC_dfs.items():
        with open(key + '_model_marginal_implied_CAC.csv', 'w') as f:
            value.to_csv(f)

In [122]:
df_convert[['join_fee', 'payment_period','monthly_price']] = df_convert[['join_fee', 'payment_period','monthly_price']].apply(pd.to_numeric)

In [123]:
df_convert['revenue'] = df_convert['join_fee']+df_convert['payment_period']*4*df_convert['monthly_price']

In [124]:
cac_technical =[]

for i in range(len(df_convert)):
    if df_convert.loc[i,'attribution_technical'] == 'bing':
        tier = df_convert.loc[i,'tier']
        cac_technical.append(CAC_dfs['attribution_technical'].loc['bing','marginal_CAC_t'+str(tier)])
    elif df_convert.loc[i,'attribution_technical'] == 'youtube':
        tier = df_convert.loc[i,'tier']
        cac_technical.append(CAC_dfs['attribution_technical'].loc['youtube','marginal_CAC_t'+str(tier)])
    elif df_convert.loc[i,'attribution_technical'] == 'facebook':
        tier = df_convert.loc[i,'tier']
        cac_technical.append(CAC_dfs['attribution_technical'].loc['facebook','marginal_CAC_t'+str(tier)])
    elif df_convert.loc[i,'attribution_technical'] == 'search':
        tier = df_convert.loc[i,'tier']
        cac_technical.append(CAC_dfs['attribution_technical'].loc['search','marginal_CAC_t'+str(tier)])
    elif df_convert.loc[i,'attribution_technical'] == 'display':
        tier = df_convert.loc[i,'tier']
        cac_technical.append(CAC_dfs['attribution_technical'].loc['display','marginal_CAC_t'+str(tier)])
    else:
        cac_technical.append(0)


df_convert['marginal_cac_technical'] = cac_technical

In [125]:
df_technical = pd.read_csv('attribution_technical_model_marginal_implied_CAC.csv')
df_convert['CLV_technical'] = df_convert['revenue']-df_convert['marginal_cac_technical']
df_convert['CLV/CAC_ratio_technical'] = df_convert['CLV_technical']/df_convert['marginal_cac_technical']
df_convert.to_csv('clv_cac_analysis.csv')
df_convert

,subid,tier,convert_TF,attribution_technical_bing,attribution_technical_display,attribution_technical_facebook,attribution_technical_youtube,attribution_technical_search,join_fee,monthly_price,...,attribution_technical,bing_credit,display_credit,facebook_credit,search_credit,youtube_credit,revenue,marginal_cac_technical,CLV_technical,CLV/CAC_ratio_technical
0,29726122,8,True,1,0,0,0,0,0.3303,4.7343,...,bing,1,0,0,0,0,19.2675,19.753086,-0.485586,-0.024583
1,24930568,5,True,0,0,1,0,0,0.6606,4.7343,...,facebook,0,0,1,0,0,19.5978,3.378378,16.219422,4.800949
2,21361112,6,True,0,0,0,0,1,0.3303,4.7343,...,search,0,0,0,1,0,19.2675,17.808219,1.459281,0.081944
3,27648682,4,True,0,0,0,0,1,0.0367,4.7343,...,search,0,0,0,1,0,37.9111,15.479876,22.431224,1.449057
4,29090912,3,True,0,0,1,0,0,0.0367,4.7343,...,facebook,0,0,1,0,0,18.9739,2.538071,16.435829,6.475717
5,21908764,3,True,0,0,1,0,0,0.0367,4.7343,...,facebook,0,0,1,0,0,18.9739,2.538071,16.435829,6.475717
6,29615706,8,True,0,0,1,0,0,0.0367,4.7343,...,facebook,0,0,1,0,0,37.9111,4.347826,33.563274,7.719553
7,26461912,8,True,0,0,0,0,1,0.0367,4.7343,...,search,0,0,0,1,0,18.9739,46.875000,-27.901100,-0.595223
8,22401037,5,True,0,0,1,0,0,0.1101,4.7343,...,facebook,0,0,1,0,0,19.0473,3.378378,15.668922,4.638001
9,28897426,7,True,0,0,0,0,1,0.0367,4.7343,...,search,0,0,0,1,0,18.9739,24.793388,-5.819488,-0.234719


In [127]:
df_convert.groupby(['tier','attribution_technical'])['CLV_technical'].count()

tier  attribution_technical
1     bing                       12
      display                     1
      facebook                  177
      search                     43
2     bing                        3
      display                     7
      facebook                 2719
      search                    869
      youtube                     1
3     bing                       27
      display                    28
      facebook                 2916
      search                   1107
      youtube                     7
4     bing                       51
      display                    26
      facebook                 3145
      search                   1430
      youtube                     8
5     bing                       53
      display                    62
      facebook                 3441
      search                   1559
      youtube                    36
6     bing                       58
      display                    64
      facebook                 4312


In [130]:
df_convert.groupby(['tier','attribution_technical'])['CLV_technical'].sum()

tier  attribution_technical
1     bing                        -71.799400
      display                      25.911100
      facebook                  -1616.877500
      search                   -11331.948300
2     bing                        147.066633
      display                     245.273833
      facebook                  92759.035081
      search                    24696.480187
      youtube                      27.911100
3     bing                        366.413700
      display                     883.039300
      facebook                  48424.896172
      search                    36630.722448
      youtube                     211.440500
4     bing                       1455.845300
      display                     922.939800
      facebook                  80421.686192
      search                    28596.902890
      youtube                    -115.648400
5     bing                       -811.146500
      display                    1337.408800
      facebook             

In [132]:
tier_channel_clv_info = df_convert.groupby(['tier','attribution_technical'])['CLV_technical'].sum()/df_convert.groupby(['tier','attribution_technical'])['CLV_technical'].count()

In [133]:
tier_channel_clv_info.to_csv('tier_channel_clv_info.csv')

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
